In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [13]:
## constants ##
score_key ="score"
ruddit_data_path = "./train/removed_redundant_ruddit_with_text.csv"
test_size_percent = 0.2 # 20% test/val data
log_dir = "./log/ensemble-nn-tuning/"
final_train_filepath = "./output/ensemble-nn-tuning/final_model"


In [3]:
ruddit_df = pd.read_csv(ruddit_data_path)
ruddit_df.shape

(5710, 3)

In [4]:
bert_1_bilstm_preds = pd.read_csv("./output/bert-1-bilstm-epoch-2-ruddit-preds.csv")[score_key]
bert_2_bilstm_preds = pd.read_csv("./output/bert-2-bilstm-epoch-2-ruddit-preds.csv")[score_key]

unitaryAI_preds_df = pd.read_csv("./output/unitaryAI-3-preds.csv")
original_preds = unitaryAI_preds_df["original"]
unbiased_preds = unitaryAI_preds_df["unbiased"]
multilingual_preds = unitaryAI_preds_df["multilingual"]

bert_1_bilstm_preds.shape, bert_2_bilstm_preds.shape, original_preds.shape, unbiased_preds.shape, multilingual_preds.shape

((5710,), (5710,), (5710,), (5710,), (5710,))

In [5]:
all_preds = np.dstack((bert_1_bilstm_preds, bert_2_bilstm_preds, original_preds, unbiased_preds, multilingual_preds))
all_preds = all_preds[0, :, :]
all_preds, all_preds.shape # can cross check all_preds[:, 0], bert_1_bilstm_preds

(array([[0.08340063, 0.10924673, 0.00136673, 0.00725068, 0.00714219],
        [0.30339277, 0.22755435, 0.00210346, 0.00850783, 0.21569288],
        [0.3519706 , 0.42682526, 0.46482915, 0.08537298, 0.03407657],
        ...,
        [0.10334039, 0.09215485, 0.00194553, 0.0008212 , 0.0027297 ],
        [0.15009415, 0.21410424, 0.00151555, 0.00102995, 0.00113789],
        [0.0714389 , 0.09716053, 0.04199111, 0.00102843, 0.00095666]]),
 (5710, 5))

In [6]:
all_inputs = keras.Input(shape=(all_preds.shape[1],), name="all_preds", dtype="float32")
x = layers.Dense(all_preds.shape[1], activation="relu")(all_inputs) # same size as input
output = layers.Dense(1, activation="tanh")(x) 

model = keras.Model(all_inputs, output)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss= 'mse',# tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError()],
)

2022-02-03 23:06:41.887532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-03 23:06:41.909002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-03 23:06:41.909303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-03 23:06:41.909766: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow wi

In [7]:
tr_ind, val_ind = train_test_split(list(range(len(all_preds))) ,test_size = test_size_percent, random_state = 23)
len(tr_ind), len(val_ind)

(4568, 1142)

In [8]:
train_data = all_preds[tr_ind, :]
val_data = all_preds[val_ind, :]
train_data.shape, val_data.shape

((4568, 5), (1142, 5))

In [9]:
y_train = ruddit_df[score_key][tr_ind]
y_val = ruddit_df[score_key][val_ind]
y_train.shape, y_val.shape

((4568,), (1142,))

In [10]:
n_epochs = 50

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq=1)

model.fit(train_data, y_train, epochs=n_epochs, validation_data=(val_data, y_val), callbacks=[tensorboard_callback])

2022-02-03 23:06:43.041102: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-02-03 23:06:43.041134: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-02-03 23:06:43.041159: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-02-03 23:06:43.041620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-02-03 23:06:43.103602: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-02-03 23:06:43.103744: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-02-03 23:06:43.142734: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
  2/143 [..............................] - ETA: 23s - loss: 0.2315 - mean_squared_error: 0.2315 - root_mean_squared_error: 0.4812 

2022-02-03 23:06:44.080661: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-02-03 23:06:44.097743: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-02-03 23:06:44.097782: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-02-03 23:06:44.261674: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-02-03 23:06:44.262255: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-02-03 23:06:44.276061: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 91 callback api events and 87 activity events. 
2022-02-03 23:06:44.277849: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


 17/143 [==>...........................] - ETA: 2s - loss: 0.1997 - mean_squared_error: 0.1997 - root_mean_squared_error: 0.4469

2022-02-03 23:06:44.287729: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./log/ensemble-nn-tuning/train/plugins/profile/2022_02_03_23_06_44

2022-02-03 23:06:44.291233: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./log/ensemble-nn-tuning/train/plugins/profile/2022_02_03_23_06_44/DESKTOP-KPOCLK7.trace.json.gz
2022-02-03 23:06:44.300440: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./log/ensemble-nn-tuning/train/plugins/profile/2022_02_03_23_06_44

2022-02-03 23:06:44.302878: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./log/ensemble-nn-tuning/train/plugins/profile/2022_02_03_23_06_44/DESKTOP-KPOCLK7.memory_profile.json.gz
2022-02-03 23:06:44.317387: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: ./log/ensemble-nn-tuning/train/plugins/profile/2022_02_03_23_0

143/143 [==============================] - 2s 11ms/step - loss: 0.1137 - mean_squared_error: 0.1137 - root_mean_squared_error: 0.3372 - val_loss: 0.0589 - val_mean_squared_error: 0.0589 - val_root_mean_squared_error: 0.2427
Epoch 2/50
143/143 [==============================] - 1s 9ms/step - loss: 0.0527 - mean_squared_error: 0.0527 - root_mean_squared_error: 0.2296 - val_loss: 0.0502 - val_mean_squared_error: 0.0502 - val_root_mean_squared_error: 0.2241
Epoch 3/50
143/143 [==============================] - 1s 9ms/step - loss: 0.0486 - mean_squared_error: 0.0486 - root_mean_squared_error: 0.2205 - val_loss: 0.0476 - val_mean_squared_error: 0.0476 - val_root_mean_squared_error: 0.2182
Epoch 4/50
143/143 [==============================] - 1s 9ms/step - loss: 0.0466 - mean_squared_error: 0.0466 - root_mean_squared_error: 0.2158 - val_loss: 0.0456 - val_mean_squared_error: 0.0456 - val_root_mean_squared_error: 0.2134
Epoch 5/50
143/143 [==============================] - 1s 9ms/step - loss: 

In [14]:
model.save(final_train_filepath)

2022-02-03 23:15:09.715616: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./output/ensemble-nn-tuning/final_model/assets
